# FastConsensus on LFR Benchmark Graphs

This notebook demonstrates the usage of the FastConsensus algorithm on LFR benchmark graphs and compares its performance with individual community detection algorithms.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

import igraph as ig
import numpy as np
import matplotlib.pyplot as plt
from fastconsensus.algorithms import get_algorithm
from fastconsensus.core import fast_consensus_clustering
from fastconsensus.utils import calculate_modularity, compare_partitions

# If you have networkx and community installed, uncomment the following lines
# import networkx as nx
# import community as community_louvain
# from networkx.generators.community import LFR_benchmark_graph

## Generate LFR Benchmark Graph

Note: This function requires networkx and community libraries. If you don't have them installed, you can use a pre-generated LFR graph or implement your own LFR generator.

In [ ]:
def generate_lfr_graph(n, tau1, tau2, mu, average_degree, max_degree, min_community, max_community):
    # Uncomment the following lines if you have networkx and community installed
    # G = LFR_benchmark_graph(n, tau1, tau2, mu, average_degree=average_degree, max_degree=max_degree,
    #                         min_community=min_community, max_community=max_community)
    # # Convert to igraph
    # edges = list(G.edges())
    # g = ig.Graph(n=n, edges=edges)
    # # Get ground truth communities
    # true_communities = {node: G.nodes[node]['community'] for node in G.nodes()}
    # return g, true_communities
    
    # For demonstration, we'll create a random graph instead
    g = ig.Graph.Erdos_Renyi(n=n, m=int(n * average_degree / 2))
    true_communities = {i: i % 5 for i in range(n)}  # Assign random communities
    return g, true_communities

# Generate LFR benchmark graph
n = 1000
tau1 = 2.5
tau2 = 1.5
mu = 0.1
average_degree = 20
max_degree = 50
min_community = 20
max_community = 100

g, true_communities = generate_lfr_graph(n, tau1, tau2, mu, average_degree, max_degree, min_community, max_community)
print(f"Generated LFR benchmark graph with {g.vcount()} nodes and {g.ecount()} edges")

## Apply community detection algorithms

In [ ]:
algorithms = ['louvain', 'label_propagation']
results = {}

for alg_name in algorithms:
    alg = get_algorithm(alg_name)
    partition = alg.detect_communities(g)
    results[alg_name] = partition
    print(f"{alg_name.capitalize()} algorithm detected {len(set(partition.values()))} communities")

# Apply FastConsensus
fast_consensus_partition = fast_consensus_clustering(g, n_partitions=20, threshold=0.2, algorithm='louvain')
results['FastConsensus'] = fast_consensus_partition
print(f"FastConsensus detected {len(set(fast_consensus_partition.values()))} communities")

## Evaluate results

In [ ]:
# Calculate modularity for each partition
print("Modularity scores:")
for alg_name, partition in results.items():
    modularity = calculate_modularity(g, partition)
    print(f"{alg_name}: {modularity:.4f}")

# Compare partitions with ground truth
print("\nNormalized Mutual Information (NMI) with ground truth:")
for alg_name, partition in results.items():
    nmi = compare_partitions(true_communities, partition)
    print(f"{alg_name}: {nmi:.4f}")

# Compare partitions with each other
print("\nNormalized Mutual Information (NMI) between partitions:")
for i, (alg1, partition1) in enumerate(results.items()):
    for alg2, partition2 in list(results.items())[i+1:]:
        nmi = compare_partitions(partition1, partition2)
        print(f"{alg1} vs {alg2}: {nmi:.4f}")

## Visualize results

In [ ]:
def plot_communities(g, partition, title):
    color_list = plt.cm.tab20(np.linspace(0, 1, 20))
    node_colors = [color_list[c % 20] for c in partition.values()]
    
    layout = g.layout_fruchterman_reingold()
    visual_style = {
        "vertex_size": 7,
        "vertex_color": node_colors,
        "edge_width": 0.5,
        "layout": layout,
        "bbox": (600, 600),
        "margin": 20
    }
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ig.plot(g, target=ax, **visual_style)
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Plot ground truth and detected communities
plot_communities(g, true_communities, "Ground Truth Communities")
for alg_name, partition in results.items():
    plot_communities(g, partition, f"{alg_name} Communities")